In [2]:
import cv2
import numpy as np 
import os

In [3]:
#dataprep
dataset_path=r'C:\Users\Arun\ML\data/'
faceData=[]
labels=[]
classid=0
name_map={}
for f in os.listdir(dataset_path):
    if f.endswith('.npy'):
        #loading the file
        name_map[classid]=f[:-4]
        dataItem=np.load(dataset_path + f)
        #print(dataItem.shape)#x value
        faceData.append(dataItem)
        m=dataItem.shape[0]
        #yvalue
        target=classid*np.ones((m,))
        classid+=1
        labels.append(target)
        
        
#print(labels)
X=np.concatenate(faceData,axis=0)
Y=np.concatenate(labels,axis=0).reshape((-1,1))

In [4]:
print(X.shape,Y.shape)
print(name_map)

(102, 30000) (102, 1)
{0: 'gr', 1: 'isha'}


In [7]:
#prediction algo using knn
def distance(p,q):
    return np.sqrt(np.sum((p-q)**2))
def knn(x,y,xt,k=5):
    m=x.shape[0]
    dlist=[]
    for i in range(m):
        d=distance(x[i],xt)
        dlist.append((d,y[i]))
    dlist=sorted(dlist)
    dlist=np.array(dlist[:k])
    labels=dlist[:,1]
    labels,counts=np.unique(labels,return_counts=True)
    idx=counts.argmax()
    pred=labels[idx]
    #print(labels,counts)
    return int(pred)

model2=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
cam=cv2.VideoCapture(0)
offset=20
while True:
    success, image = cam.read()
    if not success:
        print('Reading Camera Failed')
        break
    
    faces = model2.detectMultiScale(image, 1.3, 5)  # hyperparameters
     #render a box around each face and predict its face
    for f in faces:
        x, y, w, h = f 
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        cropped_face = image[y - offset:y + h + offset, x - offset:x + w + offset]  # crop and save largest face    
        cropped_face = cv2.resize(cropped_face, (100, 100))
        
        
        #predict name using knn
        class_predicted=knn(X,Y,cropped_face.flatten())
        name_predicted=name_map[class_predicted]
        
        cv2.putText(image,name_predicted,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.imshow("prediction window",image)    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
# Release the camera object and destroy the windows
cam.release()
cv2.destroyAllWindows()